# 数分割問題

与えられた変数のリストを, 要素の和が等しくなるよう 2 つのリストに分割する. 

In [1]:
from ortools.sat.python import cp_model

In [2]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, numbers, variable):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variable = variable
        self.__solution_count = 0
        self.__n_sets = len(variable[0])

    def on_solution_callback(self):
        self.__solution_count += 1
        for s_id in range(self.__n_sets):
            print("[", end=" ")
            for n_id, n in enumerate(numbers):
                if self.value(self.__variable[n_id][s_id]) != 1:
                    continue
                print(f"{n}", end=" ")
            print("]", end=" ")
        print()

    @property
    def solution_count(self):
        return self.__solution_count

In [3]:
class Model:
    def __init__(self, numbers: list[int], n_partition=2):
        self.numbers = numbers
        self.n_partition = n_partition
        self.sets = list(range(self.n_partition))

        if sum(self.numbers) % self.n_partition != 0:
            print("実行不能です(リストの総和が分割数で割り切れない)")
            return

        self.mean = sum(self.numbers) // self.n_partition

        self.model = cp_model.CpModel()
        self.x = [[self.model.new_bool_var(f"{n_id}_th_number_{n}_assigned_to_set_{s_id}") for s_id in self.sets] for n_id, n in enumerate(self.numbers)]

        for n_id, n in enumerate(self.numbers):
            self.model.add_exactly_one([self.x[n_id][s_id] for s_id in self.sets])

        for s_id in self.sets:
            self.model.add(sum([n * self.x[n_id][s_id] for n_id, n in enumerate(self.numbers)]) == self.mean)

    def solve(self):
        solver = cp_model.CpSolver()
        solution_printer = SolutionPrinter(self.numbers, self.x)
        solver.parameters.enumerate_all_solutions = True
        status = solver.solve(self.model, solution_printer)

In [4]:
numbers = [6, 9, 19, 13, 17]
model = Model(numbers)
model.solve()

[ 19 13 ] [ 6 9 17 ] 
[ 6 9 17 ] [ 19 13 ] 


In [5]:
numbers = [3, 20, 13, 6, 30, 40, 73]
model = Model(numbers)

実行不能です(リストの総和が分割数で割り切れない)
